In [1]:
import os, glob, io, zipfile, webbrowser
import pandas as pd
import geopandas as gpd
from shapely.geometry import LineString, Point
from pathlib import Path
import folium
from folium.plugins import MarkerCluster

In [6]:
def get_buckets():
    FOLDER = Path("../data/gtfs_bus")  # or change to another working path
    print("FOLDER exists?", FOLDER.exists())

    ### Verify the paths found in FOLDER
    zip_paths = sorted(FOLDER.glob("gtfs_*.zip"))
    print("Found:", [p.name for p in zip_paths])
    assert zip_paths, f"No GTFS zips found in {FOLDER}/gtfs_*.zip"

    # Set the pattern of the zipped filenames
    ZIP_PATTERN = "gtfs_*.zip"
    REQUIRED_FILES = ["shapes.txt", "stops.txt", "routes.txt", "trips.txt"]
    buckets = {k: [] for k in REQUIRED_FILES}

    zips = sorted(glob.glob(os.path.join(FOLDER, ZIP_PATTERN)))
    assert zips, f"No GTFS zips found in {FOLDER}/{ZIP_PATTERN}"

    for zp in zips:
        feed_name = os.path.splitext(os.path.basename(zp))[0]  # e.g., 'gtfs_m'
        with zipfile.ZipFile(zp) as z:
            names = set(z.namelist())
            for fn in REQUIRED_FILES:
                if fn in names:
                    df = pd.read_csv(z.open(fn), dtype=str, low_memory=False)
                    df["borough_feed"] = feed_name
                    buckets[fn].append(df)
                else:
                    print(f"[WARN] {fn} missing in {feed_name}")
                    
    return buckets

In [7]:
buckets = get_buckets()

FOLDER exists? True
Found: ['gtfs_b.zip', 'gtfs_busco.zip', 'gtfs_bx.zip', 'gtfs_m.zip', 'gtfs_q.zip', 'gtfs_si.zip']


In [10]:
def concat_buckets() -> pd.DataFrame:
    # concat and normalize dtypes
    shapes = pd.concat(buckets["shapes.txt"], ignore_index=True)
    stops  = pd.concat(buckets["stops.txt"],  ignore_index=True)
    routes = pd.concat(buckets["routes.txt"], ignore_index=True)
    trips  = pd.concat(buckets["trips.txt"],  ignore_index=True)
    
    return shapes, stops, routes, trips

In [11]:
shapes, stops, routes, trips = concat_buckets()

In [20]:
routes.head()

,route_id,agency_id,route_short_name,route_long_name,route_desc,route_type,route_color,route_text_color,borough_feed,route_url
0,B1,MTA NYCT,B1,Bay Ridge - Manhattan Beach,via 86th St / Ocean Pkwy,3,00AEEF,FFFFFF,gtfs_b,NaN
1,B11,MTA NYCT,B11,Sunset Park - Midwood,via 49th & 50th St / Avenue J,3,006CB7,FFFFFF,gtfs_b,NaN
2,B12,MTA NYCT,B12,Lefferts Gardens - East New York,via Clarkson Av / Empire Blvd / East New York Av,3,6CBE45,FFFFFF,gtfs_b,NaN
3,B13,MTA NYCT,B13,Spring Creek - Wyckoff Hospital,via Crescent St / Jamaica Av / Wyckoff Av,3,FAA61A,FFFFFF,gtfs_b,NaN
4,B14,MTA NYCT,B14,Spring Creek - Crown Heights,via Sutter Av / Pitkin Av,3,00AEEF,FFFFFF,gtfs_b,NaN


In [21]:
trips.head()

,route_id,service_id,trip_id,trip_headsign,direction_id,block_id,shape_id,borough_feed
0,B82+,EN_O5-Weekday,EN_O5-Weekday-028500_SBS82_901,SELECT BUS BENSNHRST BAY 38 via FLATLNDS,1,37486831,SBS821519,gtfs_b
1,B82+,EN_O5-Weekday,EN_O5-Weekday-034800_SBS82_901,SELECT BUS SPRING CRK SEAVIEW via KINGS,0,37486831,SBS821510,gtfs_b
2,B82+,EN_O5-Weekday,EN_O5-Weekday-039800_SBS82_913,SELECT BUS BENSNHRST BAY 38 via FLATLNDS,1,37486843,SBS821519,gtfs_b
3,B82+,EN_O5-Weekday,EN_O5-Weekday-047300_SBS82_913,SELECT BUS SPRING CRK SEAVIEW via KINGS,0,37486843,SBS821510,gtfs_b
4,B82+,EN_O5-Weekday,EN_O5-Weekday-029700_SBS82_902,SELECT BUS BENSNHRST BAY 38 via FLATLNDS,1,37486832,SBS821519,gtfs_b


In [12]:
def column_casting(shapes: pd.DataFrame, stops: pd.DataFrame):
    for col in ["shape_pt_lat", "shape_pt_lon"]:
        shapes[col] = shapes[col].astype(float)
    shapes["shape_pt_sequence"] = shapes["shape_pt_sequence"].astype(int)

    stops["stop_lat"] = stops["stop_lat"].astype(float)
    stops["stop_lon"] = stops["stop_lon"].astype(float)

    # make a collision-proof shape key (shape_id can repeat across feeds)
    shapes["shape_uid"] = shapes["borough_feed"] + "_" + shapes["shape_id"]

In [13]:
shapes.head()

,shape_id,shape_pt_lat,shape_pt_lon,shape_pt_sequence,borough_feed
0,B10053,40.578329,-73.940025,10001,gtfs_b
1,B10053,40.578252,-73.940719,10002,gtfs_b
2,B10053,40.578142,-73.941669,10003,gtfs_b
3,B10053,40.578042,-73.942579,10004,gtfs_b
4,B10053,40.577996,-73.943016,10005,gtfs_b


In [14]:
stops.head()

,stop_id,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station,borough_feed
0,300000,ORIENTAL BLVD/MACKENZIE ST,NaN,40.578350,-73.940029,NaN,NaN,0,NaN,gtfs_b
1,300002,ORIENTAL BLVD/JAFFRAY ST,NaN,40.578066,-73.943029,NaN,NaN,0,NaN,gtfs_b
2,300003,ORIENTAL BLVD/HASTINGS ST,NaN,40.577909,-73.944643,NaN,NaN,0,NaN,gtfs_b
3,300004,ORIENTAL BLVD/FALMOUTH ST,NaN,40.577718,-73.946200,NaN,NaN,0,NaN,gtfs_b
4,300006,ORIENTAL BLVD/DOVER ST,NaN,40.577353,-73.949552,NaN,NaN,0,NaN,gtfs_b


In [15]:
column_casting(shapes, stops)

In [16]:
shapes.head()

,shape_id,shape_pt_lat,shape_pt_lon,shape_pt_sequence,borough_feed,shape_uid
0,B10053,40.578329,-73.940025,10001,gtfs_b,gtfs_b_B10053
1,B10053,40.578252,-73.940719,10002,gtfs_b,gtfs_b_B10053
2,B10053,40.578142,-73.941669,10003,gtfs_b,gtfs_b_B10053
3,B10053,40.578042,-73.942579,10004,gtfs_b,gtfs_b_B10053
4,B10053,40.577996,-73.943016,10005,gtfs_b,gtfs_b_B10053


In [17]:
stops.head()

,stop_id,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station,borough_feed
0,300000,ORIENTAL BLVD/MACKENZIE ST,NaN,40.578350,-73.940029,NaN,NaN,0,NaN,gtfs_b
1,300002,ORIENTAL BLVD/JAFFRAY ST,NaN,40.578066,-73.943029,NaN,NaN,0,NaN,gtfs_b
2,300003,ORIENTAL BLVD/HASTINGS ST,NaN,40.577909,-73.944643,NaN,NaN,0,NaN,gtfs_b
3,300004,ORIENTAL BLVD/FALMOUTH ST,NaN,40.577718,-73.946200,NaN,NaN,0,NaN,gtfs_b
4,300006,ORIENTAL BLVD/DOVER ST,NaN,40.577353,-73.949552,NaN,NaN,0,NaN,gtfs_b


In [18]:
def create_shapes2route(trips: pd.DataFrame, routes: pd.DataFrame):
    # Mapping for shapes and route labels (short/long name)
    # Merge trips to routes
    shape2route = (
        trips[["route_id", "shape_id", "borough_feed"]].dropna()
        .drop_duplicates(["shape_id", "borough_feed"])
        .merge(
            routes[["route_id", "route_short_name", "route_long_name", "route_color", "borough_feed"]],
            on=["route_id", "borough_feed"], how="left"
        )
    )
    shape2route["shape_uid"] = shape2route["borough_feed"] + "_" + shape2route["shape_id"]
    
    return shape2route

In [23]:
shape2route = create_shapes2route(trips, routes)
shape2route.head()

,route_id,shape_id,borough_feed,route_short_name,route_long_name,route_color,shape_uid
0,B82+,SBS821519,gtfs_b,B82-SBS,Coney Island -spring Creek Towers,6CBE45,gtfs_b_SBS821519
1,B82+,SBS821510,gtfs_b,B82-SBS,Coney Island -spring Creek Towers,6CBE45,gtfs_b_SBS821510
2,B42,B420005,gtfs_b,B42,Canarsie Pier - Rockaway Parkway Station,006CB7,gtfs_b_B420005
3,B42,B421036,gtfs_b,B42,Canarsie Pier - Rockaway Parkway Station,006CB7,gtfs_b_B421036
4,Q56,Q560593,gtfs_b,Q56,Broadway Junction - Jamaica,B933AD,gtfs_b_Q560593


In [25]:
def shapes_sorted(shapes: pd.DataFrame):
    shapes_sorted = shapes.sort_values(["shape_uid", "shape_pt_sequence"])
    lines = (
        shapes_sorted
        .groupby("shape_uid")[["shape_pt_lon", "shape_pt_lat"]]
        .apply(lambda df: LineString(df.to_numpy()))
        .to_frame("geometry")
        .reset_index()
    )
    return lines

In [26]:
lines = shapes_sorted(shapes)
lines.head()

,shape_uid,geometry
0,gtfs_b_B10053,"LINESTRING (-73.940025 40.578329, -73.940719 4..."
1,gtfs_b_B10057,"LINESTRING (-73.939761 40.578199, -73.939799 4..."
2,gtfs_b_B10061,"LINESTRING (-73.985311 40.596781, -73.985279 4..."
3,gtfs_b_B10062,"LINESTRING (-73.939761 40.578199, -73.939799 4..."
4,gtfs_b_B10063,"LINESTRING (-74.02854 40.621892, -74.028689 40..."


In [36]:
def create_routes_gdf(lines: pd.DataFrame, shape2route: pd.DataFrame):
    # Merge shapes with routes geodataframe 
    routes_gdf = gpd.GeoDataFrame(lines, geometry="geometry", crs="EPSG:4326")
    routes_gdf = (
        routes_gdf
        .merge(
            shape2route[["shape_uid", "route_id", "route_short_name", "route_long_name", "route_color", "borough_feed"]],
            on="shape_uid", how="left"
        )
    )
    
    return routes_gdf

In [38]:
routes_gdf = create_routes_gdf(lines, shape2route)
routes_gdf.head()

,shape_uid,geometry,route_id,route_short_name,route_long_name,route_color,borough_feed
0,gtfs_b_B10053,"LINESTRING (-73.94002 40.57833, -73.94072 40.5...",B1,B1,Bay Ridge - Manhattan Beach,00AEEF,gtfs_b
1,gtfs_b_B10057,"LINESTRING (-73.93976 40.5782, -73.9398 40.578...",B1,B1,Bay Ridge - Manhattan Beach,00AEEF,gtfs_b
2,gtfs_b_B10061,"LINESTRING (-73.98531 40.59678, -73.98528 40.5...",B1,B1,Bay Ridge - Manhattan Beach,00AEEF,gtfs_b
3,gtfs_b_B10062,"LINESTRING (-73.93976 40.5782, -73.9398 40.578...",B1,B1,Bay Ridge - Manhattan Beach,00AEEF,gtfs_b
4,gtfs_b_B10063,"LINESTRING (-74.02854 40.62189, -74.02869 40.6...",B1,B1,Bay Ridge - Manhattan Beach,00AEEF,gtfs_b


In [43]:
routes_gdf['route_id'].value_counts()

route_id
B44      48
B6       33
S78      23
B44+     23
B41      21
         ..
S86       1
S84       1
S81       1
BX18B     1
Q70+      1
Name: count, Length: 345, dtype: int64

In [44]:
routes_gdf['route_short_name'].value_counts()

route_short_name
B44        48
B6         33
S78        23
B44-SBS    23
B41        21
           ..
S86         1
S84         1
S81         1
Bx18B       1
Q70-SBS     1
Name: count, Length: 345, dtype: int64

In [39]:
def create_stops_gdf(stops: pd.DataFrame):
    stops_gdf = gpd.GeoDataFrame(
        stops[["stop_id", "stop_name", "stop_lat", "stop_lon", "borough_feed"]],
        geometry=gpd.points_from_xy(stops["stop_lon"], stops["stop_lat"]),
        crs="EPSG:4326"
    )
    
    return stops_gdf


In [40]:
stops_gdf = create_stops_gdf(stops)
stops_gdf.head()

,stop_id,stop_name,stop_lat,stop_lon,borough_feed,geometry
0,300000,ORIENTAL BLVD/MACKENZIE ST,40.578350,-73.940029,gtfs_b,POINT (-73.94003 40.57835)
1,300002,ORIENTAL BLVD/JAFFRAY ST,40.578066,-73.943029,gtfs_b,POINT (-73.94303 40.57807)
2,300003,ORIENTAL BLVD/HASTINGS ST,40.577909,-73.944643,gtfs_b,POINT (-73.94464 40.57791)
3,300004,ORIENTAL BLVD/FALMOUTH ST,40.577718,-73.946200,gtfs_b,POINT (-73.9462 40.57772)
4,300006,ORIENTAL BLVD/DOVER ST,40.577353,-73.949552,gtfs_b,POINT (-73.94955 40.57735)


In [42]:
def load_ace_routes():    
    ace_routes_df = pd.read_csv("../data/cleaned/routes.csv")
    ace_route_list = ace_routes_df["route"].tolist()
    return ace_route_list

ace_route_list = load_ace_routes()

In [45]:
def get_ace_filters(ace_route_list):
    # Filter routes_gdf to get ACE and non-ACE program routes
    ace_routes_gdf = routes_gdf[routes_gdf["route_id"].isin(ace_route_list)]
    non_ace_routes_gdf = routes_gdf[~routes_gdf["route_id"].isin(ace_route_list)]
    
    return ace_routes_gdf, non_ace_routes_gdf

In [49]:
ace_routes_gdf, non_ace_routes_gdf = get_ace_filters(ace_route_list)
ace_routes_gdf['route_id'].nunique()

41

In [54]:
def line_to_latlon_coords(geom):
        # geom is a shapely LineString or MultiLineString
        if geom.geom_type == "LineString":
            return [(lat, lon) for lon, lat in geom.coords]
        elif geom.geom_type == "MultiLineString":
            coords = []
            for part in geom.geoms:
                coords.extend([(lat, lon) for lon, lat in part.coords])
            return coords
        else:
            return []

In [55]:
palette = [
    "red","blue","green","purple","orange","darkred","lightred","mediumgreen",
    "darkblue","darkgreen","cadetblue","darkpurple","brown","pink","lightblue",
    "lightgreen","gray","navy","lightgray", "maroon", "mediumyellow"
]
color_map = {}

In [56]:
def create_map():
    m = folium.Map(tiles="cartodbpositron", zoom_start=11, prefer_canvas=True)

    # Fit to route bounds
    minx, miny, maxx, maxy = routes_gdf.total_bounds
    m.fit_bounds([[miny, minx], [maxy, maxx]])

    # Create explicit panes so stops are ABOVE routes
    folium.map.CustomPane("routes", z_index=400).add_to(m)
    folium.map.CustomPane("stops",  z_index=650).add_to(m)
    
    return m

In [57]:
def non_ace_layer(m, color_map, palette):
    non_ace_routes_layer = folium.FeatureGroup(name="Non-ACE Routes", show=True)
    for _, row in non_ace_routes_gdf.iterrows():
        route = row.get("route_id") or row.get("route_short_name") or "route"
        if route not in color_map:
            color_map[route] = palette[len(color_map) % len(palette)]
        coords = line_to_latlon_coords(row.geometry)
        if coords:
            folium.PolyLine(
                locations=coords,
                color=color_map[route],
                weight=3,
                opacity=0.7,
                tooltip=f"Route ID: {route}",
            ).add_to(non_ace_routes_layer)
    non_ace_routes_layer.add_to(m)


In [58]:
def ace_layer(m, color_map, palette):
    ace_routes_layer = folium.FeatureGroup(name="ACE Program Routes", show=True)   
    for _, row in ace_routes_gdf.iterrows():
        route = row.get("route_id") or row.get("route_short_name") or "route"
        if route not in color_map:
            color_map[route] = palette[len(color_map) % len(palette)]
        coords = line_to_latlon_coords(row.geometry)
        if coords:
            folium.PolyLine(
                locations=coords,
                color=color_map[route],
                weight=4,
                opacity=0.9,
                tooltip=f"ACE Route ID: {route}",
            ).add_to(ace_routes_layer)
    ace_routes_layer.add_to(m)

In [ ]:
def stops_layer(m, color_map, palette):
    stops_raw = folium.FeatureGroup(name="Stops (dots)", show=True)
    for _, s in stops_gdf.iterrows():
        folium.CircleMarker(
            location=[s["stop_lat"], s["stop_lon"]],
            radius=2,
            color="#111",
            fill=True,
            fill_opacity=0.8,
            opacity=0.8,
            tooltip=f"{s.get('stop_name','')} (ID: {s.get('stop_id','')})"
        ).add_to(stops_raw)


    stops_raw.add_to(m)
    folium.LayerControl(collapsed=False).add_to(m)

In [ ]:
map = create_map()
non_ace_layer(map, color_map, palette)
ace_layer(map, color_map, palette)
stops_layer(map, color_map, palette)

In [61]:
out = Path("../data/map/mta_bus_map.html").resolve()
map.save(str(out))
print(f"Wrote {out}")

Wrote C:\Users\drodr\coding\Data-Lions\data\map\mta_bus_map.html
